In [1]:
import psutil
import re
import datetime
import random
import pandas as pd

# Display all the processes as a data-frame
---
1. create a dictionary of the processes running
2. Transpose the dataframe that is created from dictionary

In [2]:
processes_data: dict = {
    p.pid: p.info for p in psutil.process_iter(["name", "username"])
}
processes = pd.DataFrame(data=processes_data).transpose()  # .reset_index()\
# .rename(columns={"index":"pid"})
processes

,username,name
1,root,systemd
2,root,kthreadd
3,root,rcu_gp
4,root,rcu_par_gp
5,root,slub_flushwq
...,...,...
682951,bhavik,python
682962,root,kworker/u16:4-events_unbound
682963,root,kworker/u16:5-ext4-rsv-conversion
682964,root,kworker/u16:6


# Find process ids of some processes
----
telegram or discord or slack or skype

In [3]:
processes[
    processes["name"].isin(["telegram-desktop", "discord", "skype", "slack"])
]  # .index             # uncomment this just to see pids

# processes[processes["name"].isin(["telegram", "discord", "skype", "slack"])]["pid"]

,username,name
615999,bhavik,slack
616002,bhavik,slack
616003,bhavik,slack
616033,bhavik,slack
616038,bhavik,slack
616058,bhavik,slack
616147,bhavik,telegram-desktop
655248,bhavik,slack


# get the process information about process: python

In [4]:
for pid in processes[processes["name"] == "python"].index:
    print("Process info:", psutil.Process(pid))

Process info: psutil.Process(pid=75880, name='python', status='sleeping', started='2023-08-19 14:21:47')
Process info: psutil.Process(pid=75881, name='python', status='sleeping', started='2023-08-19 14:21:47')
Process info: psutil.Process(pid=75882, name='python', status='sleeping', started='2023-08-19 14:21:47')
Process info: psutil.Process(pid=75891, name='python', status='sleeping', started='2023-08-19 14:21:48')
Process info: psutil.Process(pid=682951, name='python', status='running', started='22:47:11')


# find network connections (sockets ipv4 + port)
---
kind parameter can be set to different values
refer: https://psutil.readthedocs.io/en/latest/#psutil.net_connections

In [5]:
network_connections = pd.DataFrame(data=psutil.net_connections(kind="inet4"))

# converting pid from float -> int
network_connections["pid"] = network_connections["pid"].convert_dtypes(int)

network_connections

,fd,family,type,laddr,raddr,status,pid
0,52,2,1,"(10.14.0.2, 48118)","(172.253.63.188, 5228)",ESTABLISHED,5004
1,26,2,1,"(10.14.0.2, 46098)","(192.241.190.165, 443)",ESTABLISHED,5004
2,26,2,1,"(10.14.0.2, 53498)","(162.159.135.234, 443)",ESTABLISHED,399016
3,30,2,1,"(10.14.0.2, 57708)","(142.250.31.188, 5228)",ESTABLISHED,5004
4,-1,2,2,"(0.0.0.0, 55063)",(),NONE,<NA>
...,...,...,...,...,...,...,...
64,25,2,1,"(10.14.0.2, 54056)","(3.14.34.68, 443)",ESTABLISHED,616038
65,54,2,1,"(127.0.0.1, 6463)",(),LISTEN,399071
66,-1,2,2,"(0.0.0.0, 631)",(),NONE,<NA>
67,57,2,1,"(127.0.0.1, 34047)",(),LISTEN,75722


# users

In [6]:
print("Users", psutil.users())

Users [suser(name='bhavik', terminal=':0', host='localhost', started=1692419840.0, pid=2865)]


# oneshot
---
to get more information on one particular process

In [7]:
# select a random process
process = psutil.Process(pid=random.choice(processes.index))
print(process)

# print vairous info about the process
with process.oneshot():
    print("PID:", process.pid)
    print("Name:", process.name())
    print("Parent PID:", process.ppid())
    print("Status:", process.status())

    # convert timestamp to date + time
    # reference: https://psutil.readthedocs.io/en/latest/#psutil.Process.create_time
    print(
        "Create time:",
        datetime.datetime.fromtimestamp(process.create_time()).strftime(
            "%y/%m/%d %H:%H:%S"
        ),
    )

psutil.Process(pid=2750, name='tracker-miner-fs', status='sleeping', started='2023-08-19 01:38:05')
PID: 2750
Name: tracker-miner-fs
Parent PID: 2723
Status: sleeping
Create time: 23/08/19 01:01:05


# CPU and Memory info
---


In [149]:
print(f"{'CPU usage:':20}{psutil.cpu_percent():10.2f} %")
print(f"{'CPU time:':20}{psutil.cpu_times()}")
print(f"{'Memory usage:':20}{psutil.virtual_memory().percent:10.2f} %")
print(f"{'Memory free:':20}{psutil.virtual_memory().available / (2**30):10.2f} GB")
print(f"{'Disk usage:':20}{psutil.disk_usage('/').percent:10.2f} %")
print(f"{'Disk free:':20}{psutil.disk_usage('/').free / (2**30):10.2f} GB")
print(f"{'Disk partitions:':20}{psutil.disk_partitions()}")
print(f"{'Users:':20}{psutil.users()}")
print(
    f"{'Boot time:':20}{datetime.datetime.fromtimestamp(psutil.boot_time()).strftime('%y-%m-%d %H:%M:%S')}"
)

CPU usage:                6.50 %
CPU time:           scputimes(user=186015.25, nice=367.29, system=45913.7, idle=3434775.91, iowait=3968.95, irq=0.0, softirq=6910.37, steal=0.0, guest=0.0, guest_nice=0.0)
Memory usage:            63.70 %
Memory free:              5.50 GB
Disk usage:              65.50 %
Disk free:               21.40 GB
Disk partitions:    [sdiskpart(device='/dev/nvme0n1p7', mountpoint='/', fstype='ext4', opts='rw,relatime,errors=remount-ro', maxfile=255, maxpath=4096), sdiskpart(device='/dev/loop0', mountpoint='/snap/brave/256', fstype='squashfs', opts='ro,nodev,relatime,errors=continue', maxfile=256, maxpath=4096), sdiskpart(device='/dev/loop1', mountpoint='/snap/bare/5', fstype='squashfs', opts='ro,nodev,relatime,errors=continue', maxfile=256, maxpath=4096), sdiskpart(device='/dev/nvme0n1p7', mountpoint='/var/snap/firefox/common/host-hunspell', fstype='ext4', opts='ro,noexec,noatime,errors=remount-ro', maxfile=255, maxpath=4096), sdiskpart(device='/dev/loop3', mount